# Fusion2Print End-2-End Enhancement-Embedding Pipeline

**Note:** In this notebook we can experiment with 

- Finetuning hyperparameters


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import transforms
from torchvision.transforms import functional as TF
from PIL import Image
import glob
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import cv2
from pytorch_msssim import ssim
from skimage.morphology import skeletonize
import random
from torchvision import models
import torch.optim as optim
from pytorch_msssim import ssim 
import re
import torch.fft as fft
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, accuracy_score
from sklearn.metrics import pairwise_distances, euclidean_distances

In [3]:
device = "mps" # change to your gpu or cpu
torch.mps.empty_cache()

In [ ]:
def pad_and_resize(img, size=512):
    """
    Pad to square with black padding, then resize to (size,size).
    Works for both PIL.Image and torch.Tensor (C,H,W).
    """
    #  Case 1: PIL Image 
    if isinstance(img, Image.Image):

        w, h = img.size
        pad_left = pad_right = pad_top = pad_bottom = 0

        if w > h:
            diff = w - h
            pad_top = diff // 2
            pad_bottom = diff - pad_top
        else:
            diff = h - w
            pad_left = diff // 2
            pad_right = diff - pad_left

        img_padded = TF.pad(img, padding=(pad_left, pad_top, pad_right, pad_bottom), padding_mode="constant", fill=0)
        img_resized = TF.resize(img_padded, (size, size))

        pad_info = {
            "orig_size": (h, w),
            "pad": (pad_top, pad_bottom, pad_left, pad_right),
            "resized_size": (size, size),
        }

        return img_resized, pad_info

    #  Case 2: torch.Tensor (C,H,W) 
    elif isinstance(img, torch.Tensor):

        if img.ndim != 3:
            raise ValueError(f"Expected tensor of shape (C,H,W), got {img.shape}")

        _, h, w = img.shape
        pad_left = pad_right = pad_top = pad_bottom = 0

        if w > h:
            diff = w - h
            pad_top = diff // 2
            pad_bottom = diff - pad_top
        else:
            diff = h - w
            pad_left = diff // 2
            pad_right = diff - pad_left

        img_padded = TF.pad(img, padding=(pad_left, pad_top, pad_right, pad_bottom), value=0)
        img_resized = TF.resize(img_padded, (size, size))

        pad_info = {
            "orig_size": (h, w),
            "pad": (pad_top, pad_bottom, pad_left, pad_right),
            "resized_size": (size, size),
        }

        return img_resized, pad_info

    else:
        raise TypeError(f"Unsupported input type: {type(img)}. Must be PIL.Image or torch.Tensor.")

def create_valid_mask(pad_info, size=512):
    """Create binary mask (1 = valid/original area, 0 = padding)."""
    h, w = pad_info["orig_size"]
    top, bottom, left, right = pad_info["pad"]

    mask = np.zeros((h + top + bottom, w + left + right), dtype=np.uint8)
    mask[top:top+h, left:left+w] = 1

    # Resize mask to match resized image size
    mask_resized = cv2.resize(mask, (size, size), interpolation=cv2.INTER_NEAREST)
    return torch.tensor(mask_resized, dtype=torch.float32)

def undo_pad_and_resize(output, pad_info, mask=None):
    """
    Reverts network output (size×size) back to original image size
    using the valid mask from pad_and_resize.
    """
    if isinstance(output, torch.Tensor):
        output = output.detach().cpu().numpy()
        if output.ndim == 3 and output.shape[0] in [1, 3]:
            output = np.transpose(output, (1, 2, 0))

    h, w = pad_info["orig_size"]
    top, bottom, left, right = pad_info["pad"]
    padded_h, padded_w = h + top + bottom, w + left + right

    # Resize network output back to padded size
    output_resized = cv2.resize(output, (padded_w, padded_h), interpolation=cv2.INTER_CUBIC)

    if mask is None:
        mask = np.zeros((padded_h, padded_w), dtype=np.uint8)
        mask[top:top+h, left:left+w] = 1
    elif isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()

    # Crop using mask bounding box
    ys, xs = np.where(mask > 0)
    y1, y2 = ys.min(), ys.max() + 1
    x1, x2 = xs.min(), xs.max() + 1
    output_cropped = output_resized[y1:y2, x1:x2]

    # Resize to original image size
    output_final = cv2.resize(output_cropped, (w, h), interpolation=cv2.INTER_CUBIC)

    return output_final

def undo_pad_and_resize_torch(tensor, pad_info):
    """
    Undo padding and resize the tensor (B, C, H, W) back to original image size.
    Fully differentiable version.
    pad_info = {'orig_size': (H_orig, W_orig), 'pad': (left, top, right, bottom)}
    """
    h_orig, w_orig = pad_info['orig_size']
    top, bottom, left, right = pad_info["pad"]
    padded_h, padded_w = h_orig + top + bottom, w_orig + left + right

    tensor_resized = F.interpolate(tensor, size=(padded_h, padded_w), mode='bicubic', align_corners=False)

    # Remove padding explicitly using H and W axes
    _, _, H, W = tensor_resized.shape
    h_start = top
    h_end = H - bottom if bottom > 0 else H
    w_start = left
    w_end = W - right if right > 0 else W

    tensor_resized = tensor_resized[:, :, h_start:h_end, w_start:w_end]

    # Resize back to original
    tensor = F.interpolate(tensor_resized, size=(h_orig, w_orig), mode='bicubic', align_corners=False)
    return tensor

class PadResizeToTensor:
    def __init__(self, size=512):
        self.size = size

    def __call__(self, img):
        # Pad & resize (handles PIL or torch tensor)
        img_resized, pad_info = pad_and_resize(img, self.size)

        # Convert mask from pad_info (single item, not batch)
        mask = create_valid_mask(pad_info, self.size)
        mask = mask.unsqueeze(0)  # shape (1,H,W)

        # Convert tensor values to plain ints (for safe serialization)
        pad_info = {
            "orig_size": tuple(int(x) for x in pad_info["orig_size"]),
            "pad": tuple(int(x) for x in pad_info["pad"]),
            "resized_size": tuple(int(x) for x in pad_info["resized_size"]),
        }

        return TF.to_tensor(img_resized), mask, pad_info

# 1. DualEncoderFusionNet Model (6 -> 3)

In [ ]:
# Basic Conv Block
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, stride, 1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

# Encoder
class Encoder(nn.Module):
    def __init__(self, in_ch=3):
        super().__init__()
        self.enc1 = ConvBlock(in_ch, 32)
        self.enc2 = ConvBlock(32, 64, stride=2)
        self.enc3 = ConvBlock(64, 128, stride=2)
        self.enc4 = ConvBlock(128, 256, stride=2)
    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        return [e1, e2, e3, e4]

# Attention Fusion
class AttentionFusion(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Conv2d(ch * 2, ch, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, 2, 1),
            nn.Softmax(dim=1)
        )
    def forward(self, f1, f2):
        w = self.attn(torch.cat([f1, f2], dim=1))
        return f1 * w[:, 0:1] + f2 * w[:, 1:2]

# Frequency + Edge-Aware Decoder
class Decoder(nn.Module):
    def __init__(self, in_ch=256, out_ch=3):
        super().__init__()
        self.up1 = nn.ConvTranspose2d(in_ch, 128, 4, 2, 1)
        self.up2 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.up3 = nn.ConvTranspose2d(64, 32, 4, 2, 1)
        self.out = nn.Conv2d(32, out_ch, 3, 1, 1)
        self.edge_enhance = nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False)
        self.edge_enhance.weight.data.fill_(0)
        self.edge_enhance.weight.data[:, :, 1, 1] = 1 

    def forward(self, x):
        x = F.relu(self.up1(x))
        x = F.relu(self.up2(x))
        x = F.relu(self.up3(x))
        x = torch.sigmoid(self.out(x))

        # Frequency-aware enhancement: emphasize high-frequency residual
        x_fft = fft.fftshift(fft.fft2(x, norm='ortho'))
        mag = torch.log(1 + torch.abs(x_fft))
        mag = mag / (torch.max(mag) + 1e-6)
        hf_weight = 1 + 0.2 * mag
        x = x * hf_weight

        # Edge-aware enhancement
        edge = self.edge_enhance(x)
        return x + 0.1 * edge

# Full Model
class DualEncoderFusionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flash_enc = Encoder(3)
        self.nonflash_enc = Encoder(3)
        self.fusions = nn.ModuleList([
            AttentionFusion(32),
            AttentionFusion(64),
            AttentionFusion(128),
            AttentionFusion(256)
        ])
        self.decoder = Decoder(256, 3)

    def forward(self, flash, nonflash, mask):
        f1 = self.flash_enc(flash)
        f2 = self.nonflash_enc(nonflash)
        fused = [self.fusions[i](f1[i], f2[i]) for i in range(4)]
        out = self.decoder(fused[-1])

        mask_up = F.interpolate(mask, size=out.shape[2:], mode='nearest')
        return out * mask_up

fusion_model = DualEncoderFusionNet().to(device)
fusion_model.load_state_dict(torch.load('<your_path>', map_location=device))
fusion_model.eval()

# 2. UNet RGB Optimization (3 -> 1)

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.net(x)

class UNetSmall(nn.Module):
    def __init__(self):
        super().__init__()
        self.down1 = DoubleConv(3, 32)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = DoubleConv(32, 64)
        self.pool2 = nn.MaxPool2d(2)

        self.middle = DoubleConv(64, 128)

        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv2 = DoubleConv(128, 64)
        self.up1 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.conv1 = DoubleConv(64, 32)

        self.final = nn.Conv2d(32, 1, 1) # Output grayscale
        self.act = nn.Sigmoid()

    def forward(self, x):
        c1 = self.down1(x)
        p1 = self.pool1(c1)
        c2 = self.down2(p1)
        p2 = self.pool2(c2)

        m = self.middle(p2)

        u2 = self.up2(m)
        c2_cat = torch.cat([u2, c2], dim=1)
        c2_out = self.conv2(c2_cat)
        u1 = self.up1(c2_out)
        c1_cat = torch.cat([u1, c1], dim=1)
        c1_out = self.conv1(c1_cat)

        return self.act(self.final(c1_out))
    
unet_model = UNetSmall().to(device)
unet_model.load_state_dict(torch.load("<your_path>", map_location=device))
unet_model.eval()

# 3. TripletDeepHard (1 -> 128 dim)

In [ ]:
class TripletNet(nn.Module):
    def __init__(self, embedding_dim=256, pretrained=False):
        super().__init__()
        resnet = models.resnet18(pretrained=pretrained)
        resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.backbone = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4,
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(256, embedding_dim)
        )

    def forward_once(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.head(x)
        x = F.normalize(x, p=2, dim=1)
        return x
    
    def forward(self, anchor, positive, negative):
        e_a = self.forward_once(anchor)
        e_p = self.forward_once(positive)
        e_n = self.forward_once(negative)
        return e_a, e_p, e_n
    
embedding_model = TripletNet(embedding_dim=256, pretrained=True)
embedding_model = embedding_model.to(device)
embedding_model.load_state_dict(torch.load("<your_path>", map_location=device))
embedding_model.eval()

# 4. E2E Pipeline 

In [ ]:
class E2EPipeline(nn.Module):
    """
    End-to-end pipeline for contactless fingerprint embedding.
    Accepts batched tensors (flash, nonflash, mask, pad_info) from MultiSessionFingerprintDataset.
    Returns L2-normalized embeddings (B,128).
    """

    def __init__(self, fusion_model, unet_model, embedding_model, device=None):
        super().__init__()
        self.fusion = fusion_model
        self.unet = unet_model
        self.embed = embedding_model

        self.device = device

        # set models to eval by default
        self.fusion.to(self.device).eval()
        self.unet.to(self.device).eval()
        self.embed.to(self.device).eval()

    def forward(self, flash_t, nf_t, mask, pad_info, return_intermediate=False, with_grad=False):
        flash_t = flash_t.to(self.device)
        nf_t = nf_t.to(self.device)
        mask = mask.to(self.device)

        ctx = torch.enable_grad() if with_grad else torch.no_grad()
        with ctx:
            fused = self.fusion(flash_t, nf_t, mask)

            unet_out = self.unet(fused)

            unpadded_np_list = []
            if isinstance(pad_info, dict):
                pad_info = [pad_info] * unet_out.shape[0]

            for i, info in enumerate(pad_info):
                unpadded_t = undo_pad_and_resize_torch(unet_out[i:i+1], info)
                unpadded_np_list.append(unpadded_t)

            # Convert to embeddings
            emb_list = []
            for up_t in unpadded_np_list:
                up_t_resized = F.interpolate(up_t, size=(512, 512), mode='bicubic', align_corners=False)
                up_t_resized = up_t_resized.clamp(0.0, 1.0)
                up_t_resized = (up_t_resized - 0.5) / 0.5

                if hasattr(self.embed, "forward_once"):
                    emb = self.embed.forward_once(up_t_resized)
                else:
                    emb = self.embed(up_t_resized)

                emb = F.normalize(emb, p=2, dim=1)
                emb_list.append(emb)

            emb = torch.cat(emb_list, dim=0) 

        if not with_grad:  # detach only during eval/inference
            emb = emb.detach().cpu()

        if return_intermediate:
            return {
                "embedding": emb,
                "fused": fused.detach().cpu() if not with_grad else fused,
                "unet": unet_out.detach().cpu() if not with_grad else unet_out,
            }

        return emb

    def to_device(self, device):
        self.device = device
        self.fusion.to(device)
        self.unet.to(device)
        self.embed.to(device)
        return self

    def set_trainable(self, module_names_or_objs, trainable=True):
        for m in module_names_or_objs:
            if isinstance(m, str):
                mod = getattr(self, m)
            else:
                mod = m
            for p in mod.parameters():
                p.requires_grad = trainable

# Load Dataset

In [ ]:
class MultiSessionFingerprintDataset(Dataset):
    """
    Loads paired flash/nonflash fingerprint images for two sessions (S1, S2).
    Each image is processed into:
        - tensor: (3,H,W)
        - mask: (1,H,W)
        - pad_info: dict
    Returns a dict:
        {
            "s1": {"flash": t, "nonflash": t, "mask": t, "pad_info": dict},
            "s2": {"flash": t, "nonflash": t, "mask": t, "pad_info": dict},
        }
    """

    def __init__(self, root_dir, transform=None):
        self.s1_flash_dir = os.path.join(root_dir, "<path>")
        self.s1_nonflash_dir = os.path.join(root_dir, "<path>")
        self.s2_flash_dir = os.path.join(root_dir, "<path>")
        self.s2_nonflash_dir = os.path.join(root_dir, "<path>")

        self.transform = transform

        ids = [
            f for f in os.listdir(self.s1_flash_dir)
            if os.path.exists(os.path.join(self.s2_flash_dir, f))
        ]
        self.image_ids = sorted(ids)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        fname = self.image_ids[idx]

        # Load PIL images
        s1_f = Image.open(os.path.join(self.s1_flash_dir, fname)).convert("RGB")
        s1_nf = Image.open(os.path.join(self.s1_nonflash_dir, fname)).convert("RGB")
        s2_f = Image.open(os.path.join(self.s2_flash_dir, fname)).convert("RGB")
        s2_nf = Image.open(os.path.join(self.s2_nonflash_dir, fname)).convert("RGB")

        # Apply transform (PadResizeToTensor returns (tensor, mask, pad_info))
        s1_f_t, s1_f_mask, s1_f_info = self.transform(s1_f)
        s1_nf_t, s1_nf_mask, s1_nf_info = self.transform(s1_nf)
        s2_f_t, s2_f_mask, s2_f_info = self.transform(s2_f)
        s2_nf_t, s2_nf_mask, s2_nf_info = self.transform(s2_nf)

        return {
            "s1": {
                "flash": s1_f_t,
                "nonflash": s1_nf_t,
                "mask": s1_f_mask * s1_nf_mask,
                "pad_info": s1_f_info
            },
            "s2": {
                "flash": s2_f_t,
                "nonflash": s2_nf_t,
                "mask": s2_f_mask * s2_nf_mask,
                "pad_info": s2_f_info
            }
        }


In [ ]:
def custom_collate(batch):
    def stack_session(session_list):
        return {
            "flash": torch.stack([s["flash"] for s in session_list]),
            "nonflash": torch.stack([s["nonflash"] for s in session_list]),
            "mask": torch.stack([s["mask"] for s in session_list]),
            "pad_info": [s["pad_info"] for s in session_list],
        }

    s1_list = [b["s1"] for b in batch]
    s2_list = [b["s2"] for b in batch]

    return {"s1": stack_session(s1_list), "s2": stack_session(s2_list)}

In [ ]:
transform = PadResizeToTensor(512)
root_dir = "<your_path>"

dataset = MultiSessionFingerprintDataset(root_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, drop_last=True, collate_fn=custom_collate)


device = "mps"
lr = 1e-6
epochs = 10

# Create E2E pipeline
pipeline = E2EPipeline(
    fusion_model,
    unet_model,
    embedding_model,
    device=device
).to_device(device)

pipeline.train()
# Unfreeze all
pipeline.set_trainable([pipeline.fusion, pipeline.unet, pipeline.embed], True)

params = list(pipeline.fusion.parameters()) + \
         list(pipeline.unet.parameters()) + \
         list(pipeline.embed.parameters())

optimizer = torch.optim.Adam(params, lr=lr, weight_decay=1e-7)


def fine_tune_loss(emb1, emb2, margin=0.1, w_triplet=0.3, w_intra=0.3):
    """
    emb1, emb2: tensors of shape [B, D], L2-normalized
    emb1[i] and emb2[i] correspond to the same ID
    """

    # 1. Triplet (soft margin) loss
    sim_matrix = emb1 @ emb2.T  # cosine similarity [B, B]
    pos_sim = sim_matrix.diagonal().unsqueeze(1)  # [B, 1]

    # soft-margin version of triplet loss
    triplet_loss = torch.log1p(torch.exp(sim_matrix - pos_sim + margin))
    mask = ~torch.eye(sim_matrix.size(0), dtype=torch.bool, device=sim_matrix.device)
    triplet_loss = triplet_loss[mask].mean()

    # 2. Intra-cluster compactness loss
    cos_pos = F.cosine_similarity(emb1, emb2, dim=1)
    intra_dists = (1.0 - cos_pos).mean()   # average distance between positives

    
    # 3. Combined loss
    total_loss = w_triplet * triplet_loss + w_intra * intra_dists
    return total_loss


# Fine-tuning

In [ ]:
best_val_loss = float('inf')
save_path = "save_path.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

train_losses, val_losses = [], []

for epoch in range(epochs):
    # TRAINING
    pipeline.train()
    pipeline.embed.eval()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        s1, s2 = batch["s1"], batch["s2"]

        emb1 = pipeline(s1["flash"], s1["nonflash"], s1["mask"], s1["pad_info"], with_grad=True)
        emb2 = pipeline(s2["flash"], s2["nonflash"], s2["mask"], s2["pad_info"], with_grad=True)

        loss =  fine_tune_loss(emb1, emb2, margin=0.08, w_triplet=0.75, w_intra=0.25)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(params, 1.0)
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # VALIDATION
    pipeline.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
            s1, s2 = batch["s1"], batch["s2"]
            emb1 = pipeline(s1["flash"], s1["nonflash"], s1["mask"], s1["pad_info"], with_grad=False)
            emb2 = pipeline(s2["flash"], s2["nonflash"], s2["mask"], s2["pad_info"], with_grad=False)
            val_loss += fine_tune_loss(emb1, emb2, margin=0.08, w_triplet=0.75, w_intra=0.25).item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch [{epoch+1}/{epochs}]  Train Loss: {avg_train_loss:.6f} Val Loss: {avg_val_loss:.6f}")

    # SAVE BEST MODEL
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({
            "epoch": epoch + 1,
            "pipeline_state_dict": pipeline.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
        }, save_path)
        print(f"✅ Saved new best model with Val Loss: {best_val_loss:.6f}")

print(f"\n🏁 Training complete. Best Val Loss: {best_val_loss:.6f}")
print(f"Best fine-tuned pipeline saved to: {save_path}")



In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, epochs + 1), val_losses, label='Val Loss')
plt.title("Fine-tuning Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Test

In [ ]:
transform = PadResizeToTensor(512)
root_dir = "<your_path>"

dataset = MultiSessionFingerprintDataset(root_dir, transform=transform)

test_loader = DataLoader(dataset, batch_size=4, shuffle=False, drop_last=False, collate_fn=custom_collate)
pipeline = E2EPipeline(
    fusion_model,
    unet_model,
    embedding_model,
    device=device
).to_device(device)

checkpoint = torch.load("<your_path>", map_location=device)
pipeline.load_state_dict(checkpoint["pipeline_state_dict"])
pipeline.eval()

In [ ]:
s1_image_embeddings = []
s2_image_embeddings = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        s1, s2 = batch["s1"], batch["s2"]
        emb1 = pipeline(s1["flash"], s1["nonflash"], s1["mask"], s1["pad_info"], with_grad=False)
        emb2 = pipeline(s2["flash"], s2["nonflash"], s2["mask"], s2["pad_info"], with_grad=False)

        s1_image_embeddings.append(emb1.cpu().numpy())
        s2_image_embeddings.append(emb2.cpu().numpy())

image_embeddings_s1 = np.vstack(s1_image_embeddings)
image_embeddings_s2 =np.vstack(s2_image_embeddings)

### Note: Can perform required tests or plots with embeddings arrays

In [ ]:
s1 = np.array(image_embeddings_s1)
s2 = np.array(image_embeddings_s2)
N = s1.shape[0]

# Genuine scores (diagonal)
genuine_scores = np.array([np.dot(s1[i], s2[i]) / 
                           (np.linalg.norm(s1[i]) * np.linalg.norm(s2[i])) for i in range(N)])

# Imposter scores (random mismatched pairs)
num_imposters = min(1000, N*(N-1))  # sample 1000 imposter pairs
imposter_indices1 = np.random.choice(N, num_imposters)
imposter_indices2 = np.random.choice(N, num_imposters)
# ensure different pairs
mask = imposter_indices1 != imposter_indices2
imposter_indices1 = imposter_indices1[mask]
imposter_indices2 = imposter_indices2[mask]

imposter_scores = np.array([np.dot(s1[i1], s2[i2]) /
                            (np.linalg.norm(s1[i1]) * np.linalg.norm(s2[i2])) 
                            for i1, i2 in zip(imposter_indices1, imposter_indices2)])

y_scores = np.concatenate([genuine_scores, imposter_scores])
y_true = np.concatenate([np.ones(len(genuine_scores)), np.zeros(len(imposter_scores))])

# Compute metrics
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)
frr = 1 - tpr
far = fpr

# Compute EER
eer_idx = np.argmin(np.abs(frr - far))
eer = (frr[eer_idx] + far[eer_idx]) / 2
eer_threshold = thresholds[eer_idx]
print(f"Equal Error Rate (EER): {eer:.4f} at threshold {eer_threshold:.4f}")

results = [{
    "fpr": fpr,
    "tpr": tpr,
    "fnr": frr,
    "auc": roc_auc,
    "eer": eer,
    "eer_idx": eer_idx,
    "idx": 0
}]
labels_name = {0: "Model"}
colors = {0: "tab:blue"}
eps = 1e-6  # small offset to avoid log(0)

# Plot ROC and FRR vs FAR side-by-side
plt.figure(figsize=(12, 5))

# ROC (log scale x-axis)
plt.subplot(1, 2, 1)
for r in results:
    plt.plot(r["fpr"] + eps, r["tpr"], color=colors[r["idx"]],
             label=f"{labels_name[r['idx']]} (AUC={r['auc']:.3f})")
    plt.scatter(r["fpr"][r["eer_idx"]] + eps, r["tpr"][r["eer_idx"]],
                color=colors[r["idx"]], s=30)
plt.xscale('log')
plt.xlim(1e-3, 1.0)
plt.xlabel("False Acceptance Rate (FAR, log scale)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve (log scale)")
plt.legend()
plt.grid(True, linestyle=':', alpha=0.3)

# FRR vs FAR (log-log scale)
plt.subplot(1, 2, 2)
for r in results:
    plt.plot(r["fpr"] + eps, r["fnr"] + eps, color=colors[r["idx"]],
             label=f"{labels_name[r['idx']]} (EER={r['eer']*100:.2f}%)")
    plt.scatter(r["fpr"][r["eer_idx"]] + eps, r["fnr"][r["eer_idx"]] + eps,
                color=colors[r["idx"]], s=30)
plt.xscale('log')
plt.yscale('log')
plt.xlim(1e-3, 1.0)
plt.ylim(1e-3, 1.0)
plt.xlabel("False Acceptance Rate (FAR, log scale)")
plt.ylabel("False Rejection Rate (FRR, log scale)")
plt.title("FRR vs FAR (log-log scale)")
plt.legend()
plt.grid(True, linestyle=':', alpha=0.3)

plt.tight_layout()
plt.show()